In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 32 kB/s 
     |████████████████████████████████| 198 kB 44.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=084015c465d86546b23ecfbb0ffb2c557ae9e18aaa3680941beccbc16351587c
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
pip install twarc

     |████████████████████████████████| 59 kB 2.9 MB/s 
     |████████████████████████████████| 101 kB 5.4 MB/s 
  Created wheel for configobj: filename=configobj-5.0.6-py3-none-any.whl size=34547 sha256=7c29e2d53e67e01b52e2a465235487456c0956330de0d858ce90b72e0faa3ee9
  Stored in directory: /root/.cache/pip/wheels/0d/c4/19/13d74440f2a571841db6b6e0a273694327498884dafb9cf978
Successfully built configobj
  Attempting uninstall: humanize
    Found existing installation: humanize 0.5.1
    Uninstalling humanize-0.5.1:
      Successfully uninstalled humanize-0.5.1


In [ ]:
pip install cleantext

In [ ]:
pip install emoji

     |████████████████████████████████| 175 kB 5.1 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=ed30475a03e5e7eb0d5dc44c10f6c8bc24bedb7d66e560715c38663ab34faca3
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
from twarc import Twarc2, expansions
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from cleantext import clean
import emoji
from datetime import timezone, timedelta, datetime

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Twitter data collection and cleaning

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("test").master("local[*]").getOrCreate()

### Collecting twitter data using twitter API

In [ ]:
# Replace with your twitter api bearer_token
bearer_token = "YOUR_TOKEN"
client = Twarc2(bearer_token=bearer_token)
# Specify the start time in UTC for the time period you want replies from
start_time = datetime(2022, 5, 2, 0, 0, 0, 0, timezone(timedelta(hours=-5)))
# Specify the end time in UTC for the time period you want Tweets from
end_time = datetime(2022, 5, 2, 23, 59, 59, 0, timezone(timedelta(hours=-5)))
# Use stock ticker symbol as the key word to specify the query 
query = "\$AMZN -is:retweet lang:en"

In [ ]:
# Search the tweets
search_results = client.search_recent(query=query, start_time=start_time, end_time=end_time, tweet_fields=["created_at"])

# Put the search results in a spark dataframe
data = []
for page in search_results:
  # get all the information in a single JSON
  tweets = expansions.flatten(page)
  for tweet in tweets:
    time_str = tweet['created_at'][:10]
    data.append([datetime.fromisoformat(time_str).strftime("%d-%m-%Y"), tweet["text"]])

In [ ]:
# save the searching result in a spark dataframe
tweets_df = spark.createDataFrame(data=data, schema=["time", "text"])
tweets_df.head(10)

### Tweets cleaning

In [ ]:
# remove links
tweets_df_cleaned = tweets_df.withColumn('text', F.regexp_replace('text', r"http\S+", ''))
tweets_df_cleaned = tweets_df_cleaned.withColumn('text', F.regexp_replace('text', r"www.\S+", ''))

# remove mentions
tweets_df_cleaned = tweets_df_cleaned.withColumn('text', F.regexp_replace('text', "@\s*[A-Za-z0-9_]+", ''))

# remove emoji
def remove_emoji(col):
  result = emoji.demojize(col)
  return result

# lower letters, remove extra blank space and punctuation
def lower_and_no_extra_space(col):
    if col != "":
        result = clean(col, lowercase=True, extra_spaces=True, punct=True)
        return result
    else:
        return col

# remove stopwords
# Loading stop words and removing negative stop words from the list
stop_words = stopwords.words('english')
words_to_keep = ["don", "don’t", "ain", "aren", "aren’t", "couldn", "couldn’t", "didn", "didn’t", "doesn", "doesn’t",
                 "hadn", "hadn’t", "hasn", "hasn’t", "haven", "haven’t", "isn", "isn’t", "ma", "mightn", "mightn’t",
                 "mustn’t", "needn", "needn’t", "shan", "shan’t", "no", "nor", "not", "shouldn", "shouldn’t", "wasn",
                 "wasn’t", "weren", "weren’t", "won", "won’t", "wouldn", "wouldn’t"]
my_stop_words = stop_words
for word in words_to_keep:
    if word in my_stop_words:
        my_stop_words.remove(word)

# Removing stop words from the tweet
def remove_stop_words(col):
  tokens = word_tokenize(col)
  tweet_with_no_stop_words = [token for token in tokens if token not in my_stop_words]
  reformed_tweet = ' '.join(tweet_with_no_stop_words)
  return reformed_tweet

# apply the UDF to remove emoji
clean_udf = F.UserDefinedFunction(remove_emoji, T.StringType())
tweets_df_cleaned = tweets_df_cleaned.withColumn('text', clean_udf('text'))

# remove stopwords
stopwords_udf = F.UserDefinedFunction(remove_stop_words, T.StringType())
tweets_df_cleaned = tweets_df_cleaned.withColumn('text', stopwords_udf('text'))

# convert to lowercase, remove extra blank space and some punctuation
lower = F.UserDefinedFunction(lower_and_no_extra_space, T.StringType())
tweets_df_cleaned = tweets_df_cleaned.withColumn('text', lower('text'))

# remove remaining punctuation
tweets_df_cleaned = tweets_df_cleaned.withColumn('text', F.regexp_replace('text', r'[^\w\s]', ' '))

# remove remaining blank space
tweets_df_cleaned = tweets_df_cleaned.withColumn('text', F.regexp_replace('text', r'\s+', ' '))

In [ ]:
tweets_df_cleaned.head(10)

In [ ]:
# save the cleaned data in a csv file
pd_df = tweets_df_cleaned.toPandas()
pd_df.to_csv(path_or_buf="/AMZN.csv", sep=';', index=False, encoding='utf-8', mode='a+')

# Sentiment Analysis

In [ ]:
#################################################################################
## sentiment analysis model training 
#################################################################################

from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer
from pyspark.ml.feature import StringIndexer, VectorIndexer, IndexToString
from pyspark.ml import Pipeline

from pyspark.ml.classification import LinearSVC
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import NaiveBayes, GBTClassifier

from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator


train_data = spark.read.csv("/sen140_1w.csv",header='true', inferSchema='true')
# train_data.show(5)
(train_set, val_set, test_set) = train_data.randomSplit([0.9, 0.05, 0.05], seed = 123)

tokenizer = Tokenizer(inputCol="text", outputCol="words")
cv = CountVectorizer(vocabSize=2**16, inputCol="words", outputCol='cv')
idf = IDF(inputCol='cv', outputCol="features", minDocFreq=5)
label_stringIdx = StringIndexer(inputCol = "target", outputCol = "label")
lr = LogisticRegression(maxIter=50, regParam=0.8, elasticNetParam=0.01)
pipeline = Pipeline(stages=[tokenizer, cv, idf, label_stringIdx, lr])

pipelineFit = pipeline.fit(train_set)
predictions = pipelineFit.transform(val_set)
# predictions.select("text", "target", "label", "prediction").show(5)
print(predictions.show(5))

evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
accuracy = predictions.filter(predictions.label == predictions.prediction).count() / float(val_set.count())
roc_auc = evaluator.evaluate(predictions)

print("Accuracy Score: {0:.4f}".format(accuracy))
print("ROC-AUC: {0:.4f}".format(roc_auc))

In [ ]:
#################################################################################
## new data sentiment analysis scores prediction
#################################################################################

df_text = tweets_df_cleaned.na.fill("")

# predict sentiment analysis scores
text_predictions = pipelineFit.transform(df_text)
# predictions.select("text", "target", "label", "prediction").show(5)
print(text_predictions.show(10))
print("Text scores prediction succeeded.")

In [ ]:
# save prediction results to csv
df_scores = text_predictions.select("time","text","prediction")

In [ ]:
# average scores by date
# normalize scores considering count numbers

date_scores = df_scores.groupBy("time").agg(F.mean('prediction').alias("mean"),F.count('prediction').alias("count"))
normalized_scores = date_scores.withColumn("normalized", ( F.col("mean") / F.exp(1/F.col("count")) ) )
normalized_scores.show(5)
print("Average scores by date and normalization succeeded.")

In [ ]:
# save the normalized score in a csv file
pd_df = normalized_scores.toPandas()
pd_df.to_csv(path_or_buf="/score.csv", sep=';', index=False, encoding='utf-8', mode='a+')